### Essential libraries

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset
file_path = '/content/data.csv'
data = pd.read_csv(file_path)

data.head()


,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,1,0,0,40,0,0,...,False,False,False,False,False,False,False,True,False,False
1,38,89814,9,1,1,0,0,50,0,0,...,False,False,False,False,True,False,False,False,False,False
2,28,336951,12,1,1,0,0,40,0,1,...,True,False,False,False,True,False,False,False,False,False
3,44,160323,10,0,1,7688,0,40,0,1,...,False,False,False,False,True,False,False,False,False,False
4,18,103497,10,1,0,0,0,30,0,0,...,False,False,False,False,False,False,False,True,False,False


In [ ]:
X = data.drop(columns=['income'])
y = data['income']

# Split the dataset into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate Zemel Fairness and Disparate Impact
def calculate_fairness(y_true, y_pred, gender, positive_class=1):
    # Filter by gender
    y_true_female = y_true[gender == 0]
    y_pred_female = y_pred[gender == 0]
    y_true_male = y_true[gender == 1]
    y_pred_male = y_pred[gender == 1]

    # Calculate probabilities
    prob_positive_female = sum(y_pred_female == positive_class) / len(y_pred_female)
    prob_positive_male = sum(y_pred_male == positive_class) / len(y_pred_male)

    # Calculate Zemel Fairness
    zemel_fairness = prob_positive_male - prob_positive_female

    # Calculate Disparate Impact
    disparate_impact = prob_positive_female / prob_positive_male if prob_positive_male != 0 else float('inf')

    return zemel_fairness, disparate_impact

# Apply fairness calculations
gender_test = X_test['gender']
zemel_fairness, disparate_impact = calculate_fairness(y_test, y_pred, gender_test)

print(f'Zemel Fairness: {zemel_fairness}')
print(f'Disparate Impact: {disparate_impact}')


Accuracy: 0.7975841124684365
Zemel Fairness: 0.12179195904091697
Disparate Impact: 0.20782695254209144


In [ ]:
# Define features and target variable
X = data.drop(columns=['income'])
y = data['income']

# Split the dataset into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the entire dataset and calculate probabilities
y_pred = model.predict(X)
y_prob = model.predict_proba(X)[:, 1]

# Add predictions and probabilities to the dataset
data['prediction'] = y_pred
data['probability'] = y_prob

# Split data into CP and CD
CP = data[(data['gender'] == 1) & (data['income'] == 1)].sort_values(by='probability', ascending=True)
CD = data[(data['gender'] == 0) & (data['income'] == 0)].sort_values(by='probability', ascending=False)

# Exchange the first rows of CP and CD
if not CP.empty and not CD.empty:
    CP_first = CP.iloc[0].copy()
    CD_first = CD.iloc[0].copy()

    CP.iloc[0] = CD_first
    CD.iloc[0] = CP_first

# Recombine CP and CD with the rest of the data
remaining_data = data[~data.index.isin(CP.index) & ~data.index.isin(CD.index)]
new_data = pd.concat([CP, CD, remaining_data])

# Remove the prediction and probability columns
new_data = new_data.drop(columns=['prediction', 'probability'])

# Define new features and target variable
X_new = new_data.drop(columns=['income'])
y_new = new_data['income']

# Split the new dataset into training (70%) and testing (30%) sets
X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.3, random_state=42)

# Train a new Logistic Regression model on the modified dataset
model_new = LogisticRegression(max_iter=1000)
model_new.fit(X_new_train, y_new_train)

# Make predictions on the new test set
y_new_pred = model_new.predict(X_new_test)

# Calculate accuracy
new_accuracy = accuracy_score(y_new_test, y_new_pred)
print(f'New Accuracy: {new_accuracy}')

# Calculate Zemel Fairness and Disparate Impact on the modified dataset
def calculate_fairness(y_true, y_pred, gender, positive_class=1):
    # Filter by gender
    y_true_female = y_true[gender == 0]
    y_pred_female = y_pred[gender == 0]
    y_true_male = y_true[gender == 1]
    y_pred_male = y_pred[gender == 1]

    # Calculate probabilities
    prob_positive_female = sum(y_pred_female == positive_class) / len(y_pred_female)
    prob_positive_male = sum(y_pred_male == positive_class) / len(y_pred_male)

    # Calculate Zemel Fairness
    zemel_fairness = prob_positive_male - prob_positive_female

    # Calculate Disparate Impact
    disparate_impact = prob_positive_female / prob_positive_male if prob_positive_male != 0 else float('inf')

    return zemel_fairness, disparate_impact

# Apply fairness calculations
gender_new_test = X_new_test['gender']
new_zemel_fairness, new_disparate_impact = calculate_fairness(y_new_test, y_new_pred, gender_new_test)

print(f'New Zemel Fairness: {new_zemel_fairness}')
print(f'New Disparate Impact: {new_disparate_impact}')


New Accuracy: 0.7949225414590869
New Zemel Fairness: 0.037173186569803465
New Disparate Impact: 0.6228868003118809


### Bonus section

In [4]:

# Load dataset
data = pd.read_csv('/content/data.csv')

# Define features and target variable
X = data.drop(columns=['income'])
y = data['income']
sensitive_attribute = data['gender']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(X, y, sensitive_attribute, test_size=0.3, random_state=42)

# Compute weights
def compute_weights(y, sensitive):
    df = pd.DataFrame({'y': y, 'sensitive': sensitive})
    total = len(df)
    P_y = df['y'].value_counts() / total
    P_s = df['sensitive'].value_counts() / total
    P_ys = df.groupby(['y', 'sensitive']).size() / total
    weights = df.apply(lambda row: P_ys[row['y'], row['sensitive']] / (P_y[row['y']] * P_s[row['sensitive']]), axis=1)
    return weights

weights_train = compute_weights(y_train, sensitive_train)

# Train a Logistic Regression model with sample weights
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train, sample_weight=weights_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate Zemel Fairness and Disparate Impact
def calculate_fairness(y_true, y_pred, gender, positive_class=1):
    y_true_female = y_true[gender == 0]
    y_pred_female = y_pred[gender == 0]
    y_true_male = y_true[gender == 1]
    y_pred_male = y_pred[gender == 1]

    prob_positive_female = sum(y_pred_female == positive_class) / len(y_pred_female)
    prob_positive_male = sum(y_pred_male == positive_class) / len(y_pred_male)

    zemel_fairness = prob_positive_male - prob_positive_female
    disparate_impact = prob_positive_female / prob_positive_male if prob_positive_male != 0 else float('inf')

    return zemel_fairness, disparate_impact

# Apply fairness calculations
zemel_fairness, disparate_impact = calculate_fairness(y_test, y_pred, sensitive_test)

print(f'Zemel Fairness: {zemel_fairness}')
print(f'Disparate Impact: {disparate_impact}')


Accuracy: 0.803521463181601
Zemel Fairness: 0.051921994892817167
Disparate Impact: 0.5385862248962884
